# Practica 2 
### Javier Arteaga y Elena Gonzalez

Se instalan paquetes

In [ ]:
!pip install MultiHeadAttention

ERROR: Could not find a version that satisfies the requirement MultiHeadAttention (from versions: none)
ERROR: No matching distribution found for MultiHeadAttention

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Se cargan las librerias

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import MultiHeadAttention
import os, pathlib, shutil, random
from tensorflow import keras
from tensorflow.keras import layers
import torch

Se ejecuta con la gráfica en local

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


Se define la función self_attention

In [ ]:
def self_attention(input_sequence):
    output = np.zeros(shape=input_sequence.shape)
    # Itera sobre cada token en la secuencia de entrada
    for i, pivot_vector in enumerate(input_sequence):
        scores = np.zeros(shape=(len(input_sequence),))
        for j, vector in enumerate(input_sequence):
            # Computa el producto escalar (attention score) entre el
            # token que estamos tratando y cada uno de los demás tokens
            scores[j] = np.dot(pivot_vector, vector.T)
        # Escalamos por un factor de normalización y aplicamos softmax
        scores /= np.sqrt(input_sequence.shape[1])
        scores = softmax(scores)
        new_pivot_representation = np.zeros(shape=pivot_vector.shape)
        for j, vector in enumerate(input_sequence):
            # Tomamos la suma de todas los tokens
            # ponderados por los attention scores
            new_pivot_representation += vector * scores[j]
        # Esa suma es la salida
        output[i] = new_pivot_representation
    return output

Se definen los inputs y outputs y se crea una capa de atención de múltiples cabezas (MultiHeadAttention) con un número de cabezas dado por num_heads y una dimensión clave especificada por embed_dim.

In [ ]:
num_heads = 4 
embed_dim = 256
seq_length = 10 

inputs = tf.random.uniform((32, seq_length, embed_dim))
mha_layer = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
outputs = mha_layer(inputs, inputs, inputs)

**Obtenemos los datos**

Se cargan nuestros nuestros datos, donde Reuters dataset es un conjunto de noticias breves y sus temas, publicado por Reuters en 1986. Son 46 temas diferentes

In [ ]:
from keras.datasets import reuters
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

Se separan los datos de entranamiento de los datos de test, cogiendo un 80% para entrenar y el 20% restante para pruebas

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data, train_labels, val_labels = train_test_split(
    train_data, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

Se ajustan las longitudes calculando la noticia con mayor longitud y posteriormente se aplica el relleno/truncado a las otras secuencias.

In [ ]:
lengths = [len(text) for text in train_data]
max_length = round(sum(lengths) / len(lengths))

train_data = tf.keras.preprocessing.sequence.pad_sequences(train_data, maxlen=max_length, padding='post')
val_data = tf.keras.preprocessing.sequence.pad_sequences(val_data, maxlen=max_length, padding='post')
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_data, maxlen=max_length, padding='post')

Se obtiene el índice de palabras y se invierte el indice de estas para obtener los índices de cada palabra (casa-->76)

Además se decodifican las secuencias

In [ ]:
word_index = reuters.get_word_index()

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_sequences(sequences):
    return [' '.join([reverse_word_index.get(i - 3, '?') for i in sequence]) for sequence in sequences]

decoded_train_news = decode_sequences(train_data)
decoded_test_news = decode_sequences(test_data)
decoded_val_news = decode_sequences(val_data)

Se configuran los  conjuntos de datos de TensorFlow para entrenamiento, prueba y validación, ajustando el tamaño de lote y optimizando la velocidad de entrenamiento mediante la precarga asincrónica de datos utilizando prefetch.

La constante AUTOTUNE permite a TensorFlow determinar automáticamente la cantidad óptima de elementos para precargar, mejorando así la eficiencia del proceso

In [ ]:
int_train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
int_test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_labels))
int_val_ds = tf.data.Dataset.from_tensor_slices((val_data, val_labels))

batch_size = 32 
int_train_ds = int_train_ds.batch(batch_size)
int_test_ds = int_test_ds.batch(batch_size)
int_val_ds = int_val_ds.batch(batch_size)

int_train_ds = int_train_ds.prefetch(tf.data.experimental.AUTOTUNE)
int_test_dsest_ds = int_test_ds.prefetch(tf.data.experimental.AUTOTUNE)
int_val_ds = int_val_ds.prefetch(tf.data.experimental.AUTOTUNE)


Se verifica que la forma de nuetros datos es correcta

In [ ]:
for data, labels in int_val_ds.take(1):
    print(data.shape)
    print(data[0].shape)

(32, 146)
(146,)


Se crea la clase TransformerEncoder

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        # Tamaño de los vectores de los tokens de entrada
        self.embed_dim = embed_dim
        # Tamaño de la capa densa interna
        self.dense_dim = dense_dim
        # Número de attention heads
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    # El cálculo va en call()
    def call(self, inputs, mask=None):
        # La máscara que generará la capa Embedding
        # será 2D, pero la capa de atención espera
        # ser 3D o 4D, por lo que ampliamos su rango
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    # Implementamos la serialización para
    # que podamos guardar el modelo
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

Se crea la clase PositionalEmbedding

In [ ]:
class PositionalEmbedding(layers.Layer):
    # Una desventaja de las incrustaciones de posición es que
    # la longitud de la secuencia debe conocerse de antemano
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        # Prepara una capa de embedding para los índices de token.
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            # Y otro para las posiciones te tokens
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        # Agrega ambos vectores embeddings juntos
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        # Al igual que la capa de embedding,
        # esta capa debería poder generar una
        # máscara para que podamos ignorar los
        # ceros de relleno en las entradas.
        # El framework llamará automáticamente
        # al método compute_mask y la máscara
        # se propagará a la siguiente capa.
        return tf.math.not_equal(inputs, 0)

    # Implementamos la serialización para que
    # podamos guardar el modelo.
    def get_config(self):
        config = super().get_config()
        # config = super(PositionalEmbedding, self).get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

# Creacion de los Modelos

Se van a probar distintas configuraciones para ver cuál de todas es la más óptima

## Modelo 1


En este primer modelo hemos optado por un bajo número de cabezas y dimension densa, sin embargo por aumentar considerablemente el numero del embed_dim

In [ ]:
vocab_size = 100000
dense_dim = 24
embed_dim = 256
num_heads = 2

Por lo general el mejor dropout es 0.5 por lo cual hacemos pruebas con este.
Se utiliza además el optimizador rmsprop muy usado en estos tipos de ejercicios

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(46, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding_10 (P  (None, None, 150)         15021900  
 ositionalEmbedding)                                             
                                                                 
 transformer_encoder_10 (Tr  (None, None, 150)         381964    
 ansformerEncoder)                                               
                                                                 
 global_max_pooling1d_10 (G  (None, 150)               0         
 lobalMaxPooling1D)                                              
                                                                 
 dropout_14 (Dropout)        (None, 150)               0         
                                                          

Se aplican 10 épocas para ir viendo como se desenvuelve este modelo

In [ ]:
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=10)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10


225/225 [==============================] - 20s 85ms/step - loss: 2.2300 - accuracy: 0.5002 - val_loss: 1.5763 - val_accuracy: 0.6305
Epoch 2/10
225/225 [==============================] - 19s 85ms/step - loss: 1.4234 - accuracy: 0.6722 - val_loss: 1.1961 - val_accuracy: 0.7245
Epoch 3/10
225/225 [==============================] - 19s 85ms/step - loss: 1.1130 - accuracy: 0.7356 - val_loss: 1.0859 - val_accuracy: 0.7462
Epoch 4/10
225/225 [==============================] - 18s 81ms/step - loss: 0.9082 - accuracy: 0.7859 - val_loss: 1.0630 - val_accuracy: 0.7373
Epoch 5/10
225/225 [==============================] - 18s 81ms/step - loss: 0.7736 - accuracy: 0.8110 - val_loss: 1.0904 - val_accuracy: 0.7457
Epoch 6/10
225/225 [==============================] - 18s 82ms/step - loss: 0.6480 - accuracy: 0.8377 - val_loss: 1.0958 - val_accuracy: 0.7585
Epoch 7/10
225/225 [==============================] - 19s 86ms/step - loss: 0.5565 - accuracy: 0.8607 - val_loss: 1.2041 - val_accuracy: 0.7579
Epo

Nos sale una precisión asequible de 73,2% pero aún es mejorable

## Modelo 2

En este modelo hemos decido aumentar el número de cabezas a 4 y el dense_dim de 24 a 64 y reducir el embed_dim a 150 ya que en el modelo anterior este era muy elevado

In [ ]:
vocab_size = 100000
dense_dim = 64
embed_dim = 150
num_heads = 4

Se pasa a probar el optimizador Adam que es una variación del optimizador SGD

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(46, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding_9 (Po  (None, None, 150)         15021900  
 sitionalEmbedding)                                              
                                                                 


 transformer_encoder_9 (Tra  (None, None, 150)         381964    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_9 (Gl  (None, 150)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_13 (Dropout)        (None, 150)               0         
                                                                 
 dense_29 (Dense)            (None, 46)                6946      
                                                                 
Total params: 15410810 (58.79 MB)
Trainable params: 15410810 (58.79 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Se aumentan el número de épocas a 15

In [ ]:
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=15)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/15


225/225 [==============================] - 36s 154ms/step - loss: 2.3905 - accuracy: 0.4789 - val_loss: 1.5338 - val_accuracy: 0.6439
Epoch 2/15
225/225 [==============================] - 34s 150ms/step - loss: 1.3557 - accuracy: 0.6880 - val_loss: 1.0737 - val_accuracy: 0.7479
Epoch 3/15
225/225 [==============================] - 34s 152ms/step - loss: 0.9067 - accuracy: 0.7833 - val_loss: 0.9958 - val_accuracy: 0.7863
Epoch 4/15
225/225 [==============================] - 34s 153ms/step - loss: 0.6266 - accuracy: 0.8397 - val_loss: 1.0064 - val_accuracy: 0.7913
Epoch 5/15
225/225 [==============================] - 34s 151ms/step - loss: 0.4398 - accuracy: 0.8846 - val_loss: 1.1792 - val_accuracy: 0.7835
Epoch 6/15
225/225 [==============================] - 41s 183ms/step - loss: 0.3225 - accuracy: 0.9162 - val_loss: 1.2744 - val_accuracy: 0.7702
Epoch 7/15
225/225 [==============================] - 45s 201ms/step - loss: 0.2409 - accuracy: 0.9336 - val_loss: 1.2895 - val_accuracy: 0.7

Este modelo ha mejorado en un 2% al primer modelo ya que ahora nuestro accuracy es de 75,38%

## Modelo 3

Para este tercer modelo se siguen aumentando el número de cabezas y el dense_dim ya que nos ha dado resultados y el embed_dim se mantiene prácticamente igual

In [ ]:
vocab_size = 100000
dense_dim = 128
embed_dim = 200
num_heads = 6

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(46, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding_12 (P  (None, None, 200)         20029200  
 ositionalEmbedding)                                             
                                                                 
 transformer_encoder_12 (Tr  (None, None, 200)         1016128   
 ansformerEncoder)                                               
                                                                 
 global_max_pooling1d_12 (G  (None, 200)               0         
 lobalMaxPooling1D)                                              
                                                                 
 dropout_16 (Dropout)        (None, 200)               0         
                                                          

Se sigue aumentando el número de épocas para ver si se genera sobreentrenamientoo si el modelo sigue aprendiendo correctamente

In [ ]:
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=20)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/20


225/225 [==============================] - 36s 158ms/step - loss: 2.2432 - accuracy: 0.5056 - val_loss: 1.4697 - val_accuracy: 0.6400
Epoch 2/20
225/225 [==============================] - 35s 155ms/step - loss: 1.3617 - accuracy: 0.6900 - val_loss: 1.1517 - val_accuracy: 0.7140
Epoch 3/20
225/225 [==============================] - 35s 155ms/step - loss: 1.0596 - accuracy: 0.7518 - val_loss: 1.0376 - val_accuracy: 0.7496
Epoch 4/20
225/225 [==============================] - 35s 155ms/step - loss: 0.8491 - accuracy: 0.7953 - val_loss: 0.9810 - val_accuracy: 0.7629
Epoch 5/20
225/225 [==============================] - 35s 155ms/step - loss: 0.7012 - accuracy: 0.8221 - val_loss: 1.0276 - val_accuracy: 0.7618
Epoch 6/20
225/225 [==============================] - 35s 155ms/step - loss: 0.5800 - accuracy: 0.8607 - val_loss: 1.0190 - val_accuracy: 0.7691
Epoch 7/20
225/225 [==============================] - 35s 156ms/step - loss: 0.4791 - accuracy: 0.8802 - val_loss: 1.2397 - val_accuracy: 0.7

Se puede deducir que esta configuración no es del todo buena ya la precisión ha bajado a 71,1%

## Modelo 4

Para el último modelo probamos una configuración completamente distinta a las demás

In [ ]:
vocab_size = 100000
dense_dim = 32
embed_dim = 512
num_heads = 8

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = PositionalEmbedding(max_length, vocab_size, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(46, activation="softmax")(x)

model = keras.Model(inputs, outputs)
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding_13 (P  (None, None, 512)         51274752  
 ositionalEmbedding)                                             
                                                                 
 transformer_encoder_13 (Tr  (None, None, 512)         8436768   
 ansformerEncoder)                                               
                                                                 
 global_max_pooling1d_13 (G  (None, 512)               0         
 lobalMaxPooling1D)                                              
                                                                 
 dropout_17 (Dropout)        (None, 512)               0         
                                                          

Hemos reducido drásticamente el número de épocas a 5

In [ ]:
history = model.fit(int_train_ds, validation_data=int_val_ds, epochs=5)
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/5


225/225 [==============================] - 211s 939ms/step - loss: 2.0846 - accuracy: 0.5553 - val_loss: 1.3891 - val_accuracy: 0.6806
Epoch 2/5
225/225 [==============================] - 226s 1s/step - loss: 1.1560 - accuracy: 0.7349 - val_loss: 1.2651 - val_accuracy: 0.7373
Epoch 3/5
225/225 [==============================] - 325s 1s/step - loss: 0.7030 - accuracy: 0.8262 - val_loss: 1.2621 - val_accuracy: 0.7563
Epoch 4/5
225/225 [==============================] - 269s 1s/step - loss: 0.4857 - accuracy: 0.8736 - val_loss: 1.3374 - val_accuracy: 0.7663
Epoch 5/5
71/71 [==============================] - 14s 193ms/step - loss: 1.4966 - accuracy: 0.7538
Test acc: 0.754


Este modelo nos ha dado el mayor accuracy (75,4%) por lo cuál nos quedaremos con este, además es el modelo que menos tarda en ejecutarse debido al número tan reducido de épocas